In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
a = torch.randint(0, 5, (3, 4))
b = torch.randint(0, 5, (2, 4))

print(a, b)

tensor([[3, 1, 1, 3],
        [0, 0, 0, 1],
        [4, 1, 3, 2]]) tensor([[0, 4, 3, 1],
        [2, 3, 2, 2]])


In [17]:
# convert a and b to dtype=torch.float32
a = a.float()
b = b.float()


In [18]:
b1 = torch.tensor([[0, 4, 3, 1]])
b1 = b1.float()
b1.shape

torch.Size([1, 4])

In [23]:
torch.mean((a[:, None, :] - b1) * (a[:, None, :]), dim=2).transpose(0, 1)

tensor([[2.5000, 0.0000, 3.7500]])

In [22]:
print(a[:, None, :])
torch.mean((a[:, None, :] - b) * (a[:, None, :]), dim=2).transpose(0, 1)

tensor([[[3., 1., 1., 3.]],

        [[0., 0., 0., 1.]],

        [[4., 1., 3., 2.]]])


tensor([[ 2.5000,  0.0000,  3.7500],
        [ 0.7500, -0.2500,  2.2500]])

In [32]:
tensor_example = [[4.46, 7.57, 7.58]]
# tensor_example = tensor_example
tensor_example = torch.tensor(tensor_example)

# plot the softmax output
import matplotlib.pyplot as plt
import numpy as np


# def plot_softmax(tensor_example, knn_T=10):
#     tensor_example = tensor_example / knn_T
#     softmax = torch.nn.Softmax(dim=1)
#     softmax_output = softmax(tensor_example)
#     print(softmax_output)
#     plt.plot(softmax_output.detach().numpy().flatten(), 'o')
#     # shot the x axis as discrete values
#     plt.xticks(np.arange(16))
#     # plot the y from 0 to 1
#     plt.ylim(0, 1)
#     plt.show()

# plot_softmax(tensor_example, 100)
torch.softmax(tensor_example, dim=-1)

tensor([[0.0217, 0.4867, 0.4916]])

In [5]:
tokenizer = AutoTokenizer.from_pretrained("gpt2-xl")
tokenizer(" positive")

{'input_ids': [3967], 'attention_mask': [1]}

In [2]:
'''
Choose sentiment from terrible or great.

Review: i would recommend big bad love only to winger fans who have missed her since 1995 's forget paris.
Sentiment: terrible
Review:  suspenseful enough for older kids but not . 
Sentiment: great

Review: the subtle strength of elling is that it never squandering touch with the reality of the grim situation . 
Sentiment:
'''
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-hf")
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-7b-hf", torch_dtype=torch.bfloat16, use_flash_attention_2=True)

# gpt_tokenizer = AutoTokenizer.from_pretrained("gpt2-xl")
# gpt_model = AutoModelForCausalLM.from_pretrained("gpt2-xl").to('cuda')

# mistral_tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-v0.1")
# mistral_model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-v0.1").to('cuda')

You are attempting to use Flash Attention 2.0 with a model initialized on CPU. Make sure to move the model to GPU after initializing it on CPU with `model.to('cuda')`.
Loading checkpoint shards: 100%|██████████| 2/2 [00:05<00:00,  2.59s/it]


In [4]:
model = model.to('cuda')

In [11]:
gpt_tokenizer = AutoTokenizer.from_pretrained("gpt2-xl")
gpt_model = AutoModelForCausalLM.from_pretrained("gpt2-xl").to('cuda')

In [20]:
from itertools import permutations
from tqdm import tqdm
import numpy as np
from collections import Counter
instructions = "Classify the sentiment of negative and positive."
icl_samples = ["Review: is a step down for director gary fleder . \nSentiment: negative",
               "Review: the director , tom dey , had spliced together bits and pieces of midnight run and 48 hours ( and , for that matter , shrek ) . \nSentiment: positive",
                "Review: from two fatal ailments -- a dearth of vitality and a story that 's shapeless and uninflected . \nSentiment: negative",
                "Review: results that are sometimes bracing . \nSentiment: positive",
                "Review: plodding soap opera . \nSentiment: negative",
                "Review: all-star salute . \nSentiment: positive",
                "Review: fit all of pootie tang in between its punchlines . \nSentiment: negative",
                "Review: award-winning . \nSentiment: positive",
                "Review: deserve better . \nSentiment: negative",
                "Review: you actually buy into \nSentiment: positive",
                "Review: of cliches that shoplifts shamelessly from farewell-to-innocence movies like the wanderers and a bronx tale without cribbing any of their intelligence . \nSentiment: negative",
                "Review: real-life basis is , in fact , so interesting that no embellishment is \nSentiment: positive",
                "Review: to insulting the intelligence of anyone who has n't been living under a rock \nSentiment: negative",
                "Review: immensely ambitious \nSentiment: positive",
                "Review: into the modern rut of narrative banality \nSentiment: negative",
                "Review: user-friendly \nSentiment: positive"]
sample = """or doing last year 's taxes with your ex-wife . \nSentiment:"""

tokenizer.padding_side = "left"
tokenizer.truncation_side = "left"
tokenizer.pad_token = tokenizer.eos_token

prompt_ids_list = []
i = 0
# permutation = np.random.permutation(len(icl_samples))
# icl_samples = [icl_samples[i] for i in permutation]

np.random.seed(1)
for order in range(100):
    permutation = np.random.permutation(len(icl_samples))
    order = [icl_samples[i] for i in permutation]
    prompt = '\n\n'.join(order) + '\n\n' + sample
    # prompt_ids = tokenizer.encode(prompt, return_tensors="pt")
    # prompt_ids = prompt_ids.to('cuda')

    prompt_ids_list.append((prompt))
    i += 1

print('Length of prompt_ids_list: ', len(prompt_ids_list))

batch_size = 8
prompt_ids_list = [prompt_ids_list[i:i + batch_size] for i in range(0, len(prompt_ids_list), batch_size)]

# convert prompt_ids_list to dataloader
# prompt_ids_list = torch.utils.data.DataLoader(prompt_ids_list, batch_size=batch_size, shuffle=False)

outputs = []
for batch in tqdm(prompt_ids_list):
    batch = tokenizer.batch_encode_plus(batch, return_tensors="pt", padding=True, truncation=True).to('cuda')
    output = model(batch['input_ids'], return_dict=True).logits
    output = output[:, -1, :].detach().cpu()
    # outputs += tokenizer.batch_decode(torch.argmax(output, dim=-1).squeeze().tolist())
    outputs += (tokenizer.batch_decode(torch.argmax(output, dim=-1).squeeze().tolist()))
    del batch
    
print(Counter(outputs))
# prompt = '\n\n'.join(icl_samples) + '\n\n' + sample
# prompt_ids = tokenizer.encode(prompt, return_tensors="pt")
# prompt_ids = prompt_ids.to('cuda')

# output = model(prompt_ids, return_dict=True).logits
# print(output.shape)
# output = output[:, -1, :]
# tokenizer.decode(torch.argmax(output, dim=-1).squeeze().tolist())

Length of prompt_ids_list:  100


100%|██████████| 13/13 [00:03<00:00,  3.67it/s]

Counter({'positive': 97, 'negative': 3})


In [50]:
tokenizer("terrible")

{'input_ids': [1, 16403], 'attention_mask': [1, 1]}

In [24]:
prompt = """Classify the sentiment of positive and negative.
Review: i would recommend big bad love only to winger fans who have missed her since 1995 's forget paris . .
Sentiment: negative
Review: suspenseful enough for older kids but not .
Sentiment: positive
Review: another run-of-the-mill disney sequel intended for the home video market .
Sentiment: negative
Review: has never been smoother or more confident .
Sentiment: positive
Review: bad-movie .
Sentiment: negative
Review: sweetly .
Sentiment: positive
Review: like an extended dialogue exercise in retard 101 .
Sentiment: negative
Review: bouquet gives a performance that is masterly . .
Sentiment: positive"""

input1 = "Review: one of creepiest, scariest movies to come along in a long, long time, easily rivaling blair witch or the others. \nSentiment:"
input2 = "Review: good movie . \nSentiment:"

prompt_ids = gpt_tokenizer.encode(prompt, return_tensors="pt")
inputs1 = prompt + input1
inputs2 = prompt + input2

inputs = [inputs1, inputs2]
# inputs = [inputs1, inputs1]

gpt_tokenizer.pad_token = gpt_tokenizer.eos_token
input_ids = gpt_tokenizer.batch_encode_plus(inputs, return_tensors="pt", padding=True, truncation=True)['input_ids'].to('cuda')

attention_mask = input_ids.ne(gpt_tokenizer.pad_token_id).float().to('cuda')
output = gpt_model(input_ids, attention_mask=attention_mask, return_dict=True).logits

last_non_pad_indices = torch.ne(input_ids, gpt_tokenizer.pad_token_id).sum(-1) - 1
print(last_non_pad_indices)
output = output[range(output.shape[0]), last_non_pad_indices, :]

gpt_tokenizer.batch_decode(torch.argmax(output, dim=-1).squeeze().tolist())

tensor([193, 167], device='cuda:0')


[' negative', ' positive']

In [33]:
prompt = "Classify the sentiment of positive and negative.\n Review: i would recommend big bad love only to winger fans who have missed her since 1995 's forget paris.Sentiment: negative\nReview:  suspenseful enough for older kids but not . \nSentiment: positive\n"

input = "Review: the cd is not suitable for children . \nSentiment:"
# try with the mistral tokenizer and model

prompt_ids = mistral_tokenizer.encode(prompt, return_tensors="pt")
input_ids = mistral_tokenizer.encode(input, return_tensors="pt")

input_ids = torch.cat([prompt_ids, input_ids[:, 1:]], dim=-1).to('cuda')

# add 10 padding tokens to the end of the input
input_ids = torch.cat([input_ids, torch.ones((1, 10)).long().to('cuda')], dim=-1)

attention_mask = input_ids != 1

output = mistral_model(input_ids, attention_mask=attention_mask, return_dict=True).logits
print(output.shape)
output = output[:, len(input_ids[0]) - 11, :]
mistral_tokenizer.decode(torch.argmax(output, dim=-1).squeeze().tolist())

torch.Size([1, 90, 32000])


'negative'

In [22]:
mistral_tokenizer("great")

{'input_ids': [1, 1598], 'attention_mask': [1, 1]}

In [51]:
gpt_tokenizer(" terrible")

{'input_ids': [353, 5547], 'attention_mask': [1, 1]}